In [ ]:
%matplotlib inline

# Robust optimization problem.

Minimize the expectation of the weight $w(h,t)$ w.r.t. the height $h\in[500, 800]$ and
the thickness $t\in[2,10]$ while satisfying $c_{\text{stress}}(h,t)\geq 1.0$ and
$c_{\text{displacement}}(h,t)\leq 1.0$ with probability 90\% where $F$, $E$ and
$\sigma_{\text{all}}$ are random variables defined by `BeamUncertainSpace`.


In [ ]:
from __future__ import annotations

from gemseo import configure
from gemseo import configure_logger

from gemseo_umdo.scenarios.umdo_scenario import UMDOScenario
from gemseo_umdo.use_cases.beam_model.constraints import BeamConstraints
from gemseo_umdo.use_cases.beam_model.design_space import BeamDesignSpace
from gemseo_umdo.use_cases.beam_model.discipline import Beam
from gemseo_umdo.use_cases.beam_model.uncertain_space import BeamUncertainSpace

configure()
configure_logger()

scenario = UMDOScenario(
    [Beam(), BeamConstraints()],
    "MDF",
    "w",
    BeamDesignSpace(),
    BeamUncertainSpace(uniform=False),
    "Mean",
    statistic_estimation="Sampling",
    statistic_estimation_parameters={"n_samples": 200},
)
scenario.add_constraint(
    "c_stress", "Probability", greater=False, threshold=1.0, positive=True, value=0.9
)
scenario.add_constraint(
    "c_displ", "Probability", greater=True, threshold=1.0, positive=True, value=0.9
)
scenario.execute(algo="NLOPT_COBYLA", max_iter=30)

scenario.post_process("OptHistoryView", save=False, show=True)